In [ ]:
import pandas as pd
import re
import numpy as np

# Utility Functions

In [ ]:
# fixes the street formatting 
def fixAddress(address):
    pattern = r'(\d+)\s+(st|nd|rd|th)\b'
    fixed = re.sub(pattern, r'\1\2', address, flags=re.IGNORECASE)
    return fixed

# removes apartment numbers
def getStreetAddress(address):
    dash = address.find('-')
    comma = address.find(',')
    if dash != -1:
        street = address[dash+1:comma].strip()
        return street
    else:
       return address[:comma].strip()

def getPostalCode(address):
    pattern = r', V\d[A-Za-z](?: \d[A-Za-z]\d)?,'
    match = re.search(pattern, address)
    if match:
        return match.group(0)[1:-1]
    return None

street_keywords = [
    'street', 'st', 'str', 'road', 'rd', 'blvd', 'boulevard', 'ave', 'avenue', 'ln', 'lane',
    'dr', 'drive', 'way', 'court', 'ct', 'pl', 'place', 'circle', 'cir', 'crescent', 'cr', 
    'terrace', 'ter', 'parkway', 'pkwy', 'square', 'sq', 'loop', 'highway', 'hwy', 'trail', 
]

cities = ['Vancouver',
         'Surrey',
         'Langley', 'Township of Langley',
         'Richmond',
         'Burnaby',
         'Delta',
         'Pitt Meadows',
         'New Westminster',
         'White Rock',
         'Coquitlam',
         'Abbotsford',
         'Maple Ridge']

def getNeighborhood(row):
    components = [comp.strip() for comp in row['addressDetails'].split(',')]
    components_normalized = [comp.lower() for comp in components]
    city_index = next((i for i, comp in enumerate(components_normalized) if any(city.lower() in comp for city in cities)), None)
    if city_index is None:
        return None
    neighborhood = components[city_index - 1] if city_index > 0 else None

    if neighborhood:
        if any(keyword in neighborhood.lower().split(' ') for keyword in street_keywords) or any(char.isdigit() for char in neighborhood):
            neighborhood = f"General {row['city']} District"

    return neighborhood

def getPropertyType(ptype):
    if ptype in [3, 'Condo Apt', 'Apartment/Condominium']:
        return 'Condo'
    elif ptype in [4, 5, 'Duplex', 'Triplex', 'Multi Family']:
        return 'MultiFamily'
    elif ptype in [6, 'Cottage', 'Single Family Residence', 'Detached']:
        return 'SingleFamily'
    elif ptype == 8:
        return 'Unknwon'
    elif ptype in [13, 'Townhouse']:
        return 'Townhouse'
    return None

def objToInt(col):
    if pd.isna(col) or col == '':
        return np.nan
    try:
        return int(re.sub(r'\D', '', col))
    except ValueError:
        return np.nan

def objToFloat(col):
    if pd.isna(col) or col == '':
        return np.nan
    try:
        return float(re.sub(r'[^\d.]', '', col))
    except ValueError:
        return np.nan

def makeAddress(street, city, province):
    return f'{street}, {city}, {province}'

def intToObj(col):
    return object(col)

# Preprocess Raw Data

In [ ]:
select_columns = ['mlsId', 'price', 'beds', 'baths', 'address', 'sqFt', 'parking', 'taxes', 'pricePerSqFt', 'yearBuilt', 'propertyType', 'soldDate', 'listingDate', 'listingPrice']
select_columns_active = ['mlsId', 'beds', 'baths', 'address', 'sqFt', 'parking', 'taxes', 'pricePerSqFt', 'yearBuilt', 'propertyType', 'listingDate', 'listingPrice']

def process_general_listing(df, out_path, sold=True):
    df['address'] = df['address'].apply(fixAddress)
    df['address'] = df['address'].apply(getStreetAddress) + f', {city}, BC, Canada'
    df['listingPrice'] = df['listingPrice'].apply(objToFloat)
    df['taxes'] = df['taxes'].apply(objToFloat)
    df[['sqFt', 'pricePerSqFt']] = df[['sqFt', 'pricePerSqFt']].dropna().map(objToFloat)
    df['propertyType'] = df['propertyType'].apply(getPropertyType)
    if sold:
        df['price'] = df['price'].apply(objToFloat)
    df.dropna().to_csv(out_path, index=False)

def process_active_listing(file_name, city, rename_mapping, select_columns):
    file_path = f'./data/predict/{file_name}.csv'
    df = pd.read_csv(file_path, dayfirst=True)
    df = df.rename(columns=rename_mapping)[select_columns_active].dropna().drop_duplicates()
    process_general_listing(df, f'./data/predict/ungeocoded/{city.lower()}-active-listings-ungeocoded.csv', sold=False)

def process_listing(file_name, city, rename_mapping, select_columns):
    file_path = f'./data/{file_name}.csv'
    file_path_extra = f'./data/extra/{file_name}-extra.csv'
    df = pd.read_csv(file_path, parse_dates=['soldDate', 'listingDate'], dayfirst=True)
    df_extra = pd.read_csv(file_path_extra, parse_dates=['soldDate', 'listingDate'], dayfirst=True)
    df = pd.concat([df, df_extra])
    df = df.rename(columns=rename_mapping)[select_columns].dropna().drop_duplicates()
    process_general_listing(df, f'./data/ungeocoded/{city.lower()}-listings-ungeocoded.csv')

file_city_mapping = {
    'vancouver-listings': 'Vancouver',
    'surrey-listings': 'Surrey',
    'langley-listings': 'Langley',
    'richmond-listings': 'Richmond',
    'burnaby-listings': 'Burnaby',
    'delta-listings': 'Delta',
    'pittMeadows-listings': 'Pitt Meadows',
    'newWestminster-listings': 'New Westminster',
    'whiteRock-listings': 'White Rock',
    'coquitlam-listings': 'Coquitlam',
    'abbotsford-listings': 'Abbotsford',
    'mapleRidge-listings': 'Maple Ridge'
}

rename_mapping = {
    'mlsNumber': 'mlsId',
    'soldPrice': 'price',
    'bath': 'baths',
    'area': 'sqFt',
    'built': 'yearBuilt',
    'type': 'propertyType'
}
# df_cities = [process_listing(file, city, rename_mapping, select_columns) for file, city in file_city_mapping.items()]
for file, city in file_city_mapping.items():
    df1 = process_listing(file, city, rename_mapping, select_columns)

    file_comp = file.split('-')
    active_filename = f'{file_comp[0]}-active-listings'
    process_active_listing(active_filename, city, rename_mapping, select_columns_active)

# Data Cleaning + Feature Manipulations

Note: all code blocks below take place AFTER geocoding. Make sure geocoded csvs are in place before by executing geocode.ipynb first!

In [ ]:
import pandas as pd
import numpy as np
from numpy import arcsin, sqrt, cos
import glob
import os

In [ ]:
input_dir = './data/geocoded'
output_dir = './data/combined'

# input_dir = './data/predict/geocoded'
# output_dir = './data/predict/combined'

In [ ]:
csv_files = glob.glob(os.path.join(input_dir, '*geocoded.csv'))

csv_list = []
for file in csv_files:
    csv_list.append(pd.read_csv(file))

df_totals = pd.concat(csv_list).dropna()

In [ ]:
out_path = './data/combined/total.csv'
# out_path = './data/predict/total.csv'
df_totals.to_csv(out_path, index = False) #totals -> total to test clean, will combine later

In [ ]:
city_centers = pd.read_csv('./data/features/cityCenters/centers.csv')
schools = pd.read_csv('./data/features/schools/schools_total.csv')

# Calculates the haversine distance between two coordinates in meters.
def haversine(lat1, lon1, lat2, lon2):
    r = 6371 * 1000; # Earth's radius (m)
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)

    # Adapted from Wikipedia
    return 2*r*arcsin(
            sqrt(
                (1-cos(lat2 - lat1) + 
                 cos(lat2)*cos(lat1) * 
                 (1 - cos(lon2 - lon1))) / 2
                )
            )

def find_nearest_school(row):
    distances = haversine(row['lon'], row['lat'], schools['lon'].values, schools['lat'].values)
    nearest_index = np.argmin(distances)
    return pd.Series({'distanceToSchool': distances[nearest_index], 'nearestSchool': schools.iloc[nearest_index]['name']})

## Feature Manipulations (For Training + Analysis)

In [ ]:
in_path = './data/combined/total.csv'
df = pd.read_csv(in_path)

df[['listingDate', 'soldDate']] = df[['listingDate', 'soldDate']].apply(pd.to_datetime, format='mixed', dayfirst=True)
object_columns = df.select_dtypes(include='object').columns

df[object_columns] = df[object_columns].replace('', pd.NA)

# geographical features
pattern = r'^(?P<streetname>[^,]+),\s*(?P<city>[^,]+),\s*(?P<province>[^,]+),\s*(?P<country>[^,]+)$'
df[['streetAddress', 'city', 'province', 'country']] = df['address'].str.extract(pattern)
addr_details = df['addressDetails'].str.split(', ', expand=True)
df['neighborhood'] = df.apply(getNeighborhood, axis=1)
df['postalCode'] = df['addressDetails'].apply(getPostalCode)

# historical features
df['yearBuilt'] = pd.to_numeric(df['yearBuilt'], errors='coerce')
df['age'] = 2024 - df['yearBuilt']
df['daysOnMarket'] = (df['soldDate'] - df['listingDate']).dt.days
df = df[df['taxes'] > 0]
df['priceToTaxRatio'] = df['price'] / df['taxes']
df['listingYear'] = pd.to_datetime(df['listingDate']).dt.year
df['listingMonth'] = pd.to_datetime(df['listingDate']).dt.month
df['soldYear'] = pd.to_datetime(df['soldDate']).dt.year
df['soldMonth'] = pd.to_datetime(df['soldDate']).dt.month

In [ ]:
df_tmp = pd.merge(df, city_centers, on='city', how='left')

# Calculate distance to the homes' respective city centers
df['distanceToCenter'] = haversine(
    df_tmp['lon'], df_tmp['lat'],
    df_tmp['cityCenterLon'], df_tmp['cityCenterLat']
)

# Calculate the name and distance to the nearest school
df[['distanceToSchool', 'nearestSchool']] = df_tmp.apply(find_nearest_school, axis=1)

In [ ]:
df = df.drop(columns=['mlsId', 'province', 'country', 'addressDetails'])
df.dropna().to_csv('./data/combined/total_clean.csv', index=False)

## Data Cleaning + Feature Manipulations (For Predictions)
Again... done after geocoding portion.

In [ ]:
in_path = './data/predict/total.csv'
df = pd.read_csv(in_path)

df['listingDate'] = df['listingDate'].apply(pd.to_datetime, format='mixed', dayfirst=True)
object_columns = df.select_dtypes(include='object').columns
df[object_columns] = df[object_columns].replace('', pd.NA)

pattern = r'^(?P<streetname>[^,]+),\s*(?P<city>[^,]+),\s*(?P<province>[^,]+),\s*(?P<country>[^,]+)$'
df[['streetAddress', 'city', 'province', 'country']] = df['address'].str.extract(pattern)
df['neighborhood'] = df.apply(getNeighborhood, axis=1)
df['postalCode'] = df['addressDetails'].apply(getPostalCode)

df['yearBuilt'] = pd.to_numeric(df['yearBuilt'], errors='coerce')
df['age'] = 2024 - df['yearBuilt']
df = df[df['age'] >= 0.0]
df = df[df['taxes'] > 0]
df['priceToTaxRatio'] = df['listingPrice'] / df['taxes'] # we will estimate the priceToTaxRatio with the listing price
df['listingYear'] = pd.to_datetime(df['listingDate']).dt.year
df['listingMonth'] = pd.to_datetime(df['listingDate']).dt.month

In [ ]:
df_tmp = pd.merge(df, city_centers, on='city', how='left')
df['distanceToCenter'] = haversine(
    df_tmp['lon'], df_tmp['lat'],
    df_tmp['cityCenterLon'], df_tmp['cityCenterLat']
)
df[['distanceToSchool', 'nearestSchool']] = df_tmp.apply(find_nearest_school, axis=1)

In [ ]:
df = df.drop(columns=['mlsId', 'province', 'country'])
df.dropna().to_csv('./data/predict/total_clean.csv', index=False)